In [1]:
import tensorflow as tf

In [2]:
from sklearn.datasets import fetch_mldata

In [191]:
import numpy as np

In [3]:
data = fetch_mldata('mnist original')

In [9]:
x = data.data

In [10]:
y = data.target

In [11]:
x.shape, y.shape

((70000, 784), (70000,))

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

In [16]:
sss = StratifiedShuffleSplit(random_state=42, test_size=7000)
for train_index, test_index in sss.split(x,y):
    x_train, y_train = x[train_index], y[train_index]
    x_test, y_test = x[test_index], y[test_index]

print(x_train.shape, x_test.shape)

(63000, 784) (7000, 784)


In [88]:
tf.reset_default_graph()

In [184]:
x_train = x_train/255

In [58]:
from functools import partial
he_init = tf.contrib.layers.variance_scaling_initializer()
create_dense = partial(tf.layers.dense, units = 100, kernel_initializer = he_init,activation = tf.nn.elu)

In [22]:
batch_size = 50
n_epochs = 40
learning_rate = 0.001
n_features = 784

In [224]:
tf.reset_default_graph()

In [225]:
with tf.name_scope("Inputs"):
    Xa = tf.placeholder(shape = [None,n_features], dtype = tf.float32)
    Xb = tf.placeholder(shape = [None,n_features], dtype = tf.float32)
    y = tf.placeholder(shape = [None,1], dtype = tf.float32)
    threshold = tf.placeholder_with_default(input = tf.constant([0.5]), shape = [1])

In [226]:
with tf.name_scope("DNN-A"):
    hidden1 = create_dense(inputs = Xa)
    hidden2 = create_dense(inputs = hidden1)
    hidden3 = create_dense(inputs = hidden2)
    hidden4 = create_dense(inputs = hidden3)
    hidden5a = create_dense(inputs = hidden4)

In [227]:
with tf.name_scope("DNN-B"):
    hidden1 = create_dense(inputs = Xb)
    hidden2 = create_dense(inputs = hidden1)
    hidden3 = create_dense(inputs = hidden2)
    hidden4 = create_dense(inputs = hidden3)
    hidden5b = create_dense(inputs = hidden4)

In [228]:
combined_output = tf.concat([hidden5a, hidden5b], axis=1)
concat_layer = tf.layers.dense(inputs = combined_output, units = 1, kernel_initializer=he_init)

In [229]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=concat_layer)
    loss = tf.reduce_mean(xentropy)

In [230]:
with tf.name_scope("optimizer"):
    optimizer = tf.train.MomentumOptimizer(momentum = 0.9,learning_rate=learning_rate, use_nesterov=True)
    training_op = optimizer.minimize(loss)

In [231]:
with tf.name_scope("performance"):
    correct = tf.logical_not(tf.less(concat_layer,threshold))
    accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.int32))